<a href="https://colab.research.google.com/github/ipeirotis/dealing_with_data/blob/master/02-WebAPIs/A3_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Open AI API

*The hottest new programming language is English*

-- [Andrej Karpathy](https://twitter.com/karpathy/status/1617979122625712128)

In this lesson, we will use the Open AI and GPT to performa a variety of tasks.

### Setup

In [ ]:
!pip install -U -q openai

In [ ]:
import openai
import base64
import textwrap
import json
from bs4 import BeautifulSoup
import requests

key = 'KEY ON SLACK'
openai.api_key = base64.b64decode(key).decode('ascii')

## Helper function

Throughout this course, we will use OpenAI's `gpt-3.5-turbo` model and the [chat completions endpoint](https://platform.openai.com/docs/guides/gpt/chat-completions-api). If necessary, we can switch to the `gpt-4-32k` model (slower, but better and with higher context length).

This helper function will make it easier to use prompts and look at the generated outputs:

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]



In [ ]:
get_completion("Can you tell me where Stern School of Business is located?")

In [ ]:
get_completion("Do you know the undergraduate program at Stern?")

## Prompting and Responses

### Sentiment Analysis

In our scenario, we will ask GPT to infer sentiment and topics from product reviews and news articles.



In [ ]:
# This is the review that we want to analyze

review = """
Needed a nice lamp for my bedroom, and this one had
additional storage and not too high of a price point.
Got it fast.  The string to our lamp broke during the
transit and the company happily sent over a new one.
Came within a few days as well. It was easy to put
together.  I had a missing part, so I contacted their
support and they very quickly got me the missing piece!
Lumina seems to me to be a great company that cares
about their customers and products!!
"""

In [ ]:
# This is the prompt that we submit to GPT (notice that we embed the review)

prompt = f"""
What is the sentiment of the following product review below?

Review text: {review}
"""

In [ ]:
response = get_completion(prompt)
print(response)

You will notice that the reponse is good for a human, but perhaps we want to ask GPT to give us a more concise and structured response.

In [ ]:
# The hottest new programming language is English

prompt = f"""
What is the sentiment of the following product review?
Give your answer as a single word, either "positive" or "negative".

Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

In [ ]:
prompt = f"""
What is the sentiment of the following product review?
Give your answer as a single word, either "positive" or "negative".

Review text: '''I went there for dinner. Had to wait for an hour for the host
to sit us to the table. Drinks overpriced, food bland. Too much hype.'''
"""
response = get_completion(prompt)
print(response)

### Identify types of emotions

In [ ]:
prompt = f"""
Identify a list of emotions that the writer of the
following review is expressing. Include no more than
five items in the list. Format your answer as a list of
lower-case words separated by commas.

Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

In [ ]:
prompt = f"""
Identify a list of emotions that the writer of the
following review is expressing. Include no more than
five items in the list. Format your answer as a list of
lower-case words separated by commas.

Review text: '''I went there for dinner. Had to wait for an hour for the host
to sit us to the table. Drinks overpriced, food bland. Too much hype.'''
"""
response = get_completion(prompt)
print(response)

In [ ]:
emotion_list = response.split(", ")
emotion_list

### Extract product and company name from customer reviews

In [ ]:
prompt = f"""
Identify the following items from the review text:
- Item purchased by reviewer
- Company that made the item

Format your response as a JSON object with
"Item" and "Brand" as the keys.
If the information isn't present, use "unknown"
as the value.
Make your response as short as possible.

Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

In [ ]:
data = json.loads(response)
data

### Doing multiple tasks at once

In [ ]:
prompt = f"""
Identify the following items from the review text:
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks.
Format your response as a JSON object with
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown"
as the value.
Format the Anger value as a boolean.
Make your response as short as possible.

Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

In [ ]:
data = json.loads(response)
data

## Summarize stories

In [ ]:
story = """
In a recent survey conducted by the government,
public sector employees were asked to rate their level
of satisfaction with the department they work at.
The results revealed that NASA was the most popular
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings,
stating, "I'm not surprised that NASA came out on top.
It's a great place to work with amazing people and
incredible opportunities. I'm proud to be a part of
such an innovative organization."

The results were also welcomed by NASA's management team,
with Director Tom Johnson stating, "We are thrilled to
hear that our employees are satisfied with their work at NASA.
We have a talented and dedicated team who work tirelessly
to achieve our goals, and it's fantastic to see that their
hard work is paying off."

The survey also revealed that the
Social Security Administration had the lowest satisfaction
rating, with only 45% of employees indicating they were
satisfied with their job. The government has pledged to
address the concerns raised by employees in the survey and
work towards improving job satisfaction across all departments.
"""

In [ ]:
prompt = f"""
Summarize the story below, using 20 words or less.

Story: {story}
"""
response = get_completion(prompt)

# The textwrap simply allows us to print the text
# with a maximum width, so that we can read it.
print(textwrap.fill(response, width=80))

### Infer topics

In [ ]:
prompt = f"""
Determine five topics that are being discussed in the
following text. Make each item one or two words long.

Format your response as a list of items separated by commas.

Story: '''{story}'''
"""
response = get_completion(prompt)
print(response)

In [ ]:
topics = response.split(", ")
topics

## Exercise

* Use the API at https://newsapi.org to fetch the news.  * Fetch all articles about a company of your choice, from the last month, sorted by recent first (e.g., for Tesla https://newsapi.org/v2/everything?q=tesla&from=2023-05-12&sortBy=publishedAt&apiKey=API_KEY).
* Analyze the sentiment of the news and/or summarize the latest news about the company. Initially, work only with 3-5 articles, to ensure that your code works. Do not try to analyze 100 articles at once.


In [ ]:
# Helper functions
def retrieve_text_from_url(url):
    """Remove html tags from a string"""
    try:
      resp = requests.get(url)
      soup = BeautifulSoup(resp.text, "html.parser")
      return soup.get_text()
    except:
      return ""

In [ ]:
# Helper functions
def get_company_news(company, page = 1, num_articles=100):
  endpoint = 'https://newsapi.org/v2/everything'
  parameters = {
      'q' : company, # query for a company name
      'from': '2023-05-15', # retrieve articles over the last month
      'sortBy': 'publishedAt', # sort the most recent articles on top
      'apiKey' : '960f6acd257a47eba0a336ec72337d03',
      'searchIn' : 'title', # the name of the company should appear in the title of the news article
      'pageSize' : num_articles, # return at most num_articles
      'page': page # which page of results to return
  }
  resp = requests.get(endpoint, params=parameters)
  data = resp.json()
  print(data)
  if 'articles' in data:
    return data['articles']
  else:
    return []

In [ ]:
# Helper function for analyzing using ChatGPT
def analyze_article(article):
  '''
  This function takes as input a JSON object that has the same structure
  as an article coming back from NewsAPI and returns back the results
  from ChatGPT.
  '''

  article_title = article['title']
  article_text = retrieve_text_from_url(article['url'])[:5000] # keep first 5,000 characters

  prompt = f'''
  Analyze the contents of this article and return the sentiment of the article
  towards {company_to_analyze}.

  Structure the result as a JSON object,  with a Summary field,
  a Sentiment field, a Sentiment_Score and a list of topics.

  The summary should be between 10 and 40 words long.
  The sentiment should be Positive, Neutral, or Negative.
  The sentiment score should be between -1 and 1, with -1 being most negative, 1 being most positive, and 0 being neutral.

  Story: {article_title} -- {article_text}
  '''

  try:
    response = get_completion(prompt)
    data = json.loads(response)
    return data
  except: #There was an error
    return {}

Here is some code below to get you started. Try not to limit yourself to the prompt below, or even to the analysis done here. Come up with ideas and try them out. Learning is more important than the final result!

In [ ]:
company_to_analyze = 'Microsoft'

# Get the articles for the company
articles = get_company_news(company=company_to_analyze, num_articles=3)

In [ ]:
# Let's take a look at an article in the list
i = 2
articles[i]

In [ ]:
# Let's analyze the articles
for a in articles:
  print(analyze_article(a))

In [ ]:
# YOUR CODE HERE
#
# Go through all the articles and store the results
# Do some analysis of the retrieved data


## Exercises

* Use NewsAPI to get the top headlines for different languages. Ask ChatGPT to summarize the news being discussed in different languages.

* Plot the average sentiment score for a company over time. Examine the use of advanced plots (eg violin plots) to show the distribution of sentiment over time, not just the averages.